In [15]:
# importing libraries
import librosa as lb
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

In [16]:
targetLabelData = pd.read_csv('tabulatedData.csv')
targetLabelData.head()

,clipName,N,A,D,F,S,H,low,medium,high
0,1010_MTI_SAD_XX,0.727273,0.000000,0.000000,0.000000,0.272727,0.0,0.090909,0.454545,0.454545
1,1037_IWL_HAP_XX,0.000000,0.400000,0.000000,0.000000,0.000000,0.6,0.100000,0.600000,0.300000
2,1022_IWW_FEA_XX,0.200000,0.000000,0.000000,0.500000,0.300000,0.0,0.100000,0.400000,0.500000
3,1067_WSI_HAP_XX,0.300000,0.200000,0.100000,0.300000,0.100000,0.0,0.200000,0.500000,0.300000
4,1001_TIE_FEA_XX,0.181818,0.636364,0.090909,0.090909,0.000000,0.0,0.000000,0.727273,0.272727


In [3]:
# folder path
path = "ProcessedAudio"

features_data = []

# iterating through the sounds
for item in tqdm(os.listdir(path)):
    # getting sound path
    item_path = os.path.join(path, item)
    
    #Get the fileName
    fileName = item.split('.wav')[0]

    # result array to store data
    result = np.array([])
    
    # load sound
    y, sr = lb.load(item_path, sr = None)

    n_fft = 128
    
    # Spectrogram Magnitude
    S = np.abs(lb.stft(y, n_fft = n_fft))
    # Mel-frequency cepstral coefficient (MFCC)
    mfcc = lb.feature.mfcc(y = y, sr = sr, n_fft = n_fft)
    mfcc = np.mean(mfcc.T, axis = 0)
    result = np.hstack((result,mfcc))
    # print(len(result))

    # MEL spectrogram
    mel = lb.feature.melspectrogram(y = y, sr = sr)
    mel = np.mean(mel.T, axis = 0)
    result = np.hstack((result,mel))
    # print(len(result))
    
    # chromagram
    chroma = lb.feature.chroma_stft(S = S, sr = sr, n_fft=n_fft)
    chroma = np.mean(chroma.T, axis = 0)
    result = np.hstack((result,chroma))
    # print(len(result))

    # Spectral Contrast
    contrast = lb.feature.spectral_contrast(S = S, sr = sr, n_fft = n_fft)
    contrast = np.mean(contrast.T, axis = 0)
    result = np.hstack((result, contrast))
    # print(len(result))

    # harmonic component - for tonnetz
    h = lb.effects.harmonic(y)

    # tonnetz
    t = lb.feature.tonnetz(y = h, sr = sr)
    t = np.mean(t.T, axis = 0)
    result = np.hstack((result,t))
    # print(len(result))

    features_data.append([fileName,*result])

  0%|          | 0/7442 [00:00<?, ?it/s]/Users/takehararyoutarou/anaconda3/envs/mlVenv/lib/python3.10/site-packages/librosa/feature/spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)
  0%|          | 1/7442 [00:01<4:08:00,  2.00s/it]/Users/takehararyoutarou/anaconda3/envs/mlVenv/lib/python3.10/site-packages/librosa/feature/spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)
  0%|          | 10/7442 [00:03<17:27,  7.10it/s] /Users/takehararyoutarou/anaconda3/envs/mlVenv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=512 is too large for input signal of length=418
  warnings.warn(


In [9]:
mfcc_feats = [f'mfcc_{i}' for i in range(20)]
mel_feats = [f'mel_{i}' for i in range(20,128)]
chroma_feats = [f'chroma_{i}' for i in range (128,160)]
spect_feats = [f'spect_{i}' for i in range(160,167)]
tone_feats = [f'tonnets_{i}' for i in range(167,173)]

column_names = ['clipName']+ mfcc_feats + mel_feats + chroma_feats + spect_feats + tone_feats
features_df = pd.DataFrame(features_data, columns=column_names)

In [10]:
features_df.head()

,clipName,mfcc_0,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,...,spect_163,spect_164,spect_165,spect_166,tonnets_167,tonnets_168,tonnets_169,tonnets_170,tonnets_171,tonnets_172
0,1022_ITS_ANG_XX,-625.058655,-4.994524,-5.040599,31.118151,-11.813456,-14.570557,0.724647,-9.413414,-3.972650,...,9.217098,11.644959,15.893591,10.260955,0.019559,0.050362,-0.078460,-0.014718,0.001586,0.009466
1,1037_ITS_ANG_XX,-648.157227,-10.045857,7.429339,20.802429,-12.531164,-11.790744,-3.735406,-6.268928,-9.658600,...,10.222218,11.192162,13.460297,10.360918,0.016312,0.036811,0.006748,0.011417,0.027077,0.032486
2,1060_ITS_NEU_XX,-565.523743,-29.746906,5.289134,20.724148,-6.095077,-11.757398,0.216888,-9.311358,-4.887557,...,8.128854,10.070826,12.236355,10.058037,0.034823,-0.020027,0.010998,-0.091110,0.014165,0.003219
3,1075_ITS_NEU_XX,-556.091248,-15.240954,11.779830,20.665905,-9.193678,-11.914318,0.438259,-7.635978,-7.933719,...,9.781172,10.263756,12.968909,9.800903,-0.016201,-0.010273,-0.058400,-0.064868,0.010537,0.001605
4,1073_IOM_DIS_XX,-622.589111,-13.248747,4.246189,30.263844,-10.529772,-10.335563,1.289759,-11.116067,-5.185853,...,8.199897,9.780688,13.834202,9.830638,-0.008683,0.013556,-0.009265,-0.082829,0.017282,0.040066


In [17]:
featuresAndLabels = pd.merge(features_df, targetLabelData, on=['clipName'], how='inner')

In [19]:
print(len(featuresAndLabels))
featuresAndLabels.head()

7442


,clipName,mfcc_0,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,...,tonnets_172,N,A,D,F,S,H,low,medium,high
0,1022_ITS_ANG_XX,-625.058655,-4.994524,-5.040599,31.118151,-11.813456,-14.570557,0.724647,-9.413414,-3.972650,...,0.009466,0.000000,0.454545,0.545455,0.0,0.0,0.0,0.000000,0.272727,0.727273
1,1037_ITS_ANG_XX,-648.157227,-10.045857,7.429339,20.802429,-12.531164,-11.790744,-3.735406,-6.268928,-9.658600,...,0.032486,0.500000,0.200000,0.200000,0.1,0.0,0.0,0.000000,0.300000,0.700000
2,1060_ITS_NEU_XX,-565.523743,-29.746906,5.289134,20.724148,-6.095077,-11.757398,0.216888,-9.311358,-4.887557,...,0.003219,0.900000,0.000000,0.000000,0.0,0.1,0.0,0.000000,0.200000,0.800000
3,1075_ITS_NEU_XX,-556.091248,-15.240954,11.779830,20.665905,-9.193678,-11.914318,0.438259,-7.635978,-7.933719,...,0.001605,0.909091,0.000000,0.090909,0.0,0.0,0.0,0.181818,0.090909,0.727273
4,1073_IOM_DIS_XX,-622.589111,-13.248747,4.246189,30.263844,-10.529772,-10.335563,1.289759,-11.116067,-5.185853,...,0.040066,0.300000,0.000000,0.600000,0.0,0.1,0.0,0.200000,0.200000,0.600000


In [20]:
save_loc = 'featuresAndLabels.csv'

featuresAndLabels.to_csv(save_loc, index = False)